In [39]:
import pandas as pd

Read in data

In [42]:
allergies = pd.read_csv('allergies.csv')
careplans = pd.read_csv('careplans.csv')
# claims = pd.read_csv('claims.csv')
conditions = pd.read_csv('conditions.csv')
encounters = pd.read_csv('encounters.csv')
immunizations = pd.read_csv('immunizations.csv')
medications = pd.read_csv('medications.csv')
observations = pd.read_csv('observations.csv')
patients = pd.read_csv('patients.csv')
procedures = pd.read_csv('procedures.csv')

Drop Columns

In [43]:
column_names = ['START','STOP']
allergies_clean = allergies.drop(column_names, axis=1)
careplans_clean = careplans.drop(column_names, axis=1)
conditions_clean = conditions.drop(column_names, axis=1)
medications_clean = medications.drop(column_names, axis=1)

# Drop unnecessary patient information
patients = patients.drop('ssn', axis=1)
patients = patients.drop('passport', axis=1)
patients = patients.drop('first', axis=1)
patients = patients.drop('last', axis=1)
patients = patients.drop('maiden', axis=1)
patients = patients.drop('suffix', axis=1)
patients = patients.drop('prefix', axis=1)
## Do we need address/birthplace? could be geographically interesting ig?
patients = patients.drop('drivers', axis=1) ## WHAT IS DRIVERS? don't think we need but do more research to make sure

# drop diagnosis from claims as all entries are empty
# claims = claims.drop('DIAGNOSIS', axis=1)
# claims = claims.drop('ORGANIZATION', axis=1) # deleted this as all organizations are temp organizations

column_names = ['DATE']
observations_clean = observations.drop(column_names, axis=1)
immunizations_clean = immunizations.drop(column_names, axis=1)
encounters_clean = encounters.drop(column_names, axis=1)

Clean up observations

In [ ]:
observations_clean

,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS
0,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,8302-2,Body Height,166.03,cm
1,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,29463-7,Body Weight,54.42,kg
2,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,39156-5,Body Mass Index,19.74,kg/m2
3,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,8480-6,Systolic Blood Pressure,139.0,mmHg
4,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,8462-4,Diastolic Blood Pressure,89.0,mmHg
...,...,...,...,...,...,...
79669,8b9de29c-34c9-405f-840c-71b5210cf9e1,c7455496-e0a6-4a24-ae0d-72e41772c202,2571-8,Triglycerides,120.0,mg/dL
79670,8b9de29c-34c9-405f-840c-71b5210cf9e1,c7455496-e0a6-4a24-ae0d-72e41772c202,18262-6,Low Density Lipoprotein Cholesterol,86.0,mg/dL
79671,8b9de29c-34c9-405f-840c-71b5210cf9e1,c7455496-e0a6-4a24-ae0d-72e41772c202,2085-9,High Density Lipoprotein Cholesterol,65.0,mg/dL
79672,8b9de29c-34c9-405f-840c-71b5210cf9e1,NaN,273724008,Quality adjusted life years,49.124509075732604,years


Code to change observations to have one row per patient

In [44]:
observations_clean['VALUE'] = pd.to_numeric(observations_clean['VALUE'], errors='coerce')

# Pivot table with mean aggregation
observations_pivot= observations_clean.pivot_table(index=observations_clean.index, columns='DESCRIPTION', values='VALUE', fill_value=0, aggfunc='mean')

observations_pivot['PATIENT'] = observations_clean['PATIENT']
observations_pivot = observations_pivot.groupby('PATIENT').sum().reset_index()


In [49]:
careplans_pivot = pd.get_dummies(careplans_clean['DESCRIPTION'])
careplans_pivot['PATIENT'] = careplans_clean['PATIENT']
careplans_pivot = careplans_pivot.groupby('PATIENT').sum().reset_index()


In [51]:
conditions_pivot = pd.get_dummies(conditions_clean['DESCRIPTION'])
conditions_pivot['PATIENT'] = conditions_clean['PATIENT']
conditions_pivot = conditions_pivot.groupby('PATIENT').sum().reset_index()

In [ ]:
immunizations_pivot = pd.get_dummies(immunizations_clean['DESCRIPTION'])
immunizations_pivot['PATIENT'] = immunizations_clean['PATIENT']
immunizations_pivot = immunizations_pivot.groupby('PATIENT').sum().reset_index()

In [ ]:
conditions_pivot

Add suffixes to columns

In [ ]:
patients = patients.rename(columns={'patient': 'PATIENT'})

def add_suffix(df, suffix):
    renamed_columns = {}
    for col_name in df.columns:
        if col_name != 'PATIENT':
            renamed_columns[col_name] = col_name + '_' + suffix
        else:
            renamed_columns[col_name] = col_name
    return df.rename(columns=renamed_columns)


allergies_clean = add_suffix(allergies_clean, 'ALLERGIES')
careplans_clean = add_suffix(careplans_clean, 'CAREPLANS')
# claims_clean = add_suffix(claims, 'CLAIMS')
conditions_clean = add_suffix(conditions_clean, 'CONDITIONS')
encounters_clean = add_suffix(encounters_clean, 'ENCOUNTERS')
immunizations_clean = add_suffix(immunizations_clean, 'IMMUNIZATIONS')
medications_clean = add_suffix(medications_clean, 'MEDICATIONS')
observations_pivot = add_suffix(observations_pivot, 'OBSERVATIONS')
procedures_clean = add_suffix(procedures_clean, 'PROCEDURES')


Split up encounters to add stuff

In [ ]:
total_rows = len(encounters_clean)
rows_per_section = total_rows // 8

section1 = encounters_clean.iloc[:rows_per_section]
section2 = encounters_clean.iloc[rows_per_section:2*rows_per_section]
section3 = encounters_clean.iloc[2*rows_per_section:3*rows_per_section]
section4 = encounters_clean.iloc[3*rows_per_section:4*rows_per_section]
section5 = encounters_clean.iloc[4*rows_per_section:5*rows_per_section]
section6 = encounters_clean.iloc[5*rows_per_section:6*rows_per_section]
section7 = encounters_clean.iloc[6*rows_per_section:7*rows_per_section]
section8 = encounters_clean.iloc[7*rows_per_section:]

section1 = add_suffix(section1, '1')
section2 = add_suffix(section2, '2')
section3 = add_suffix(section3, '3')
section4 = add_suffix(section4, '4')
section5 = add_suffix(section5, '5')
section6 = add_suffix(section6, '6')
section7 = add_suffix(section7, '7')
section8 = add_suffix(section8, '8')

Merge datasets

In [ ]:
def merge_datasets(conditions_spec):
    
    merged_df = pd.merge(conditions_spec, patients, on='PATIENT', how='left')
    
    # # Check if there are any matched patient IDs
    # if not merged_df.empty:
    #     print("Matched patient IDs found.")
    #     # Optionally, print or inspect the matched patient IDs
    #     print("Matched patient IDs:", merged_df['PATIENT'].unique())
    # else:
    #     print("No matched patient IDs found.")
    
    merged_df = pd.merge(merged_df, allergies_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, careplans_clean, on='PATIENT', how='left')
    merged_df = pd.merge(merged_df, procedures_clean, on='PATIENT', how='left')
    
    # # Check if there are any matched patient IDs
    # if not merged_df.empty:
    #     print("2 Matched patient IDs found.")
    #     # Optionally, print or inspect the matched patient IDs
    #     print("2 Matched patient IDs:", merged_df['PATIENT'].unique())
    # else:
    #     print("2 No matched patient IDs found.")
    
    # print("onto encounter merges")
    
    # merged_df = pd.merge(merged_df, section1, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section2, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section3, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section4, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section5, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section6, on='PATIENT', how='left')
    # merged_df = pd.merge(merged_df, section7, on='PATIENT', how='left')
    
    print("3 merges left to go")
    
    merged_df = pd.merge(merged_df, immunizations_clean, on='PATIENT', how='left')
    
    print("1")
    
    merged_df = pd.merge(merged_df, medications_clean, on='PATIENT', how='left')
    print("2")

    merged_df = pd.merge(merged_df, observations_pivot, on='PATIENT', how='left')
    print("3")
    
    return merged_df

Separate data based on disease

CHANGE TO USE DESCRIPTIONS INSTEAD OF CODES

In [ ]:
diabetes_codes =  [44054006]
# merged_datasets = {}
# merged_data = []

conditions_spec = conditions_clean[conditions_clean['CODE_CONDITIONS'].isin(diabetes_codes)]

# print(f"{conditions_spec=}")

merged_diabetes = merge_datasets(conditions_spec)

print("merged diabetes")

3 merges left to go
1
2
3
merged diabetes


In [ ]:
merged_diabetes

,PATIENT,ENCOUNTER_CONDITIONS,CODE_CONDITIONS,birthdate,deathdate,marital,race,ethnicity,gender,birthplace,...,Sodium_OBSERVATIONS,Soybean IgE Ab in Serum_OBSERVATIONS,Systolic Blood Pressure_OBSERVATIONS,Total Cholesterol_OBSERVATIONS,Total score [MMSE]_OBSERVATIONS,Triglycerides_OBSERVATIONS,Urea Nitrogen_OBSERVATIONS,Walnut IgE Ab in Serum_OBSERVATIONS,Wheat IgE Ab in Serum_OBSERVATIONS,White oak IgE Ab in Serum_OBSERVATIONS
0,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006,1939-08-09,NaN,S,white,irish,F,Bellingham MA US,...,1549.0,0.0,1307.0,1926.0,0.0,1358.0,165.0,0.0,0.0,0.0
1,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006,1939-08-09,NaN,S,white,irish,F,Bellingham MA US,...,1549.0,0.0,1307.0,1926.0,0.0,1358.0,165.0,0.0,0.0,0.0
2,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006,1939-08-09,NaN,S,white,irish,F,Bellingham MA US,...,1549.0,0.0,1307.0,1926.0,0.0,1358.0,165.0,0.0,0.0,0.0
3,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006,1939-08-09,NaN,S,white,irish,F,Bellingham MA US,...,1549.0,0.0,1307.0,1926.0,0.0,1358.0,165.0,0.0,0.0,0.0
4,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006,1939-08-09,NaN,S,white,irish,F,Bellingham MA US,...,1549.0,0.0,1307.0,1926.0,0.0,1358.0,165.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552205,7d5a76ac-774d-4bc1-8e66-13104086f467,c4a6c5be-131c-4caf-bc26-9a83e78e7580,44054006,1971-03-02,NaN,M,white,swedish,F,Middleborough MA US,...,414.0,0.0,468.0,645.0,0.0,485.0,40.0,0.0,0.0,0.0
552206,7d5a76ac-774d-4bc1-8e66-13104086f467,c4a6c5be-131c-4caf-bc26-9a83e78e7580,44054006,1971-03-02,NaN,M,white,swedish,F,Middleborough MA US,...,414.0,0.0,468.0,645.0,0.0,485.0,40.0,0.0,0.0,0.0
552207,7d5a76ac-774d-4bc1-8e66-13104086f467,c4a6c5be-131c-4caf-bc26-9a83e78e7580,44054006,1971-03-02,NaN,M,white,swedish,F,Middleborough MA US,...,414.0,0.0,468.0,645.0,0.0,485.0,40.0,0.0,0.0,0.0
552208,7d5a76ac-774d-4bc1-8e66-13104086f467,c4a6c5be-131c-4caf-bc26-9a83e78e7580,44054006,1971-03-02,NaN,M,white,swedish,F,Middleborough MA US,...,414.0,0.0,468.0,645.0,0.0,485.0,40.0,0.0,0.0,0.0


In [ ]:
conditions_spec

,PATIENT,ENCOUNTER_CONDITIONS,CODE_CONDITIONS
12,96b24072-e1fe-49cd-a22a-6dfb92c3994c,4e7beaee-50c2-4609-8a2b-b32fb3dc5a3b,44054006
21,de43eb48-496c-46d4-8c5b-be6125a38c15,febdc129-bd41-4b7c-b1e7-a4ce07931544,44054006
132,81979c08-df99-4792-b608-65fcf28f4019,f3c9bdc5-2474-4458-a2cd-3255353a21e5,44054006
347,428ab0b1-a2bd-4f08-863e-40f62b368c3f,a44e288f-8c1d-4b8a-8f47-d43b95b94ba8,44054006
505,c2cc147d-a113-4d82-8f82-4c43318d3c32,4fbdd445-d44e-4e99-8336-2001e0769417,44054006
...,...,...,...
6664,55f1a2b1-da37-476a-9bce-ca82a880863e,1021a758-5854-4175-9a8b-8c6e02d3585c,44054006
6728,59f6b849-0eb5-4102-9bd2-724bf841a58f,eb382333-3da5-459e-8eb9-98056aa9aa37,44054006
6758,eafb0264-5c27-4a92-97d3-119c67a5defb,0c5daab6-fad3-4b46-b478-190fbe8b8974,44054006
6933,7c622c0e-58e0-4abf-91bd-b29fb0237bb4,9301f0f8-3028-4a4e-aff6-52e47d7c76f7,44054006


Calculate data usage

In [ ]:
# # Get memory usage of each column
# memory_usage_per_column = merged_df.memory_usage(deep=True)

# # Sum up memory usage of all columns
# total_memory_usage = memory_usage_per_column.sum()

# print("Total memory usage of DataFrame: {} bytes".format(total_memory_usage))

NOTES FROM PHIL TO STOP KERNEL DEATH
Size of indiv. dataframes
- delete columns that we are not going to use before we merge

- read in datasets a chunk at a time
    - operate on sequential parts of the data

- if its still an issue, could get set up on cluster to compute there as well